In [1]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import math
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from pickle import dump
import utils as eda
from sklearn.neighbors import KNeighborsClassifier

In [2]:


url = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/refs/heads/main/winequality-red.csv"

df = pd.read_csv(url , sep= ";")
st.dataframe(df)
target = 'label'

2025-08-29 16:32:21.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.088 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-08-29 16:32:22.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:


# Eliminar duplicados / verificacion

total_data = df.drop_duplicates()

st.write("Shape original:", df.shape)
st.write("Shape sin duplicados:", total_data.shape)
st.write("Duplicados eliminados:", df.duplicated().sum())
st.write("¿Hay duplicados en total_data?:", total_data.duplicated().any())
st.write(total_data.head(2))

total_data = total_data.copy()
total_data['label'] = (total_data['quality'] >= 6).astype(int) #creo variable label 

# X, y
X = total_data.drop(columns=['quality', 'label'])  # 
y = total_data['label']

# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("completado")

2025-08-29 16:32:22.108 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

completado


In [4]:

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Escalado completado")

Escalado completado


In [5]:
# KNN con k fijo (ajusta k si quieres)
k = 15
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_scaled, y_train)


,n_neighbors,15
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [6]:
# Evaluación
y_pred = knn.predict(X_test_scaled)
st.write("accuracy:", accuracy_score(y_test, y_pred))
st.write("confusion_matrix:\n", confusion_matrix(y_test, y_pred))
st.write("\nclassification_report:\n", classification_report(y_test, y_pred, target_names=["not_good", "good"]))

2025-08-29 16:32:22.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.229 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.231 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [7]:
# Bucle de optimización de k
k_values = range(1, 20 + 1)
train_accuracies, test_accuracies = [], []

st.write("k\tTrain Acc\tTest Acc")
st.write("-" * 30)
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_train_pred = knn.predict(X_train_scaled)
    y_test_pred = knn.predict(X_test_scaled)
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    st.write(f"{k}\t{train_acc:.4f}\t\t{test_acc:.4f}")
    print("Proceso completado")

2025-08-29 16:32:22.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.313 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.313 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.314 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.315 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-08-29 16:32:22.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.483 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Proceso completado
Proceso completado
Proceso completado
Proceso completado


2025-08-29 16:32:22.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.589 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:22.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Proceso completado
Proceso completado
Proceso completado
Proceso completado
Proceso completado
Proceso completado


...

Proceso completado
Proceso completado
Proceso completado
Proceso completado


2025-08-29 16:32:23.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Proceso completado
Proceso completado
Proceso completado
Proceso completado
Proceso completado


2025-08-29 16:32:23.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.254 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.254 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Proceso completado


In [8]:
# Mejor k
best_k = k_values[np.argmax(test_accuracies)]
st.write(f"\nMejor k: {best_k} | Mejor accuracy en test: {max(test_accuracies):.4f}")

2025-08-29 16:32:23.261 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
# Entrenar y evaluar con mejor k
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train_scaled, y_train)
y_pred = best_knn.predict(X_test_scaled)

st.write("\nMétricas con el mejor k:")
st.write("accuracy:", accuracy_score(y_test, y_pred))
st.write("confusion_matrix:\n", confusion_matrix(y_test, y_pred))
st.write("\nclassification_report:\n", classification_report(y_test, y_pred, target_names=["not_good", "good"]))

fig = plt.figure(figsize=(10, 6))
plt.plot(k_values, train_accuracies, 'o-', label='Train Accuracy', color='blue')
plt.plot(k_values, test_accuracies, 'o-', label='Test Accuracy', color='red')
plt.axvline(x=best_k, color='green', linestyle='--', alpha=0.7, label=f'Best k={best_k}')
plt.xlabel('k (número de vecinos)')
plt.ylabel('Accuracy')
plt.title('Optimización de k en KNN (total_data)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(list(k_values))
st.pyplot(fig)

2025-08-29 16:32:23.294 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.294 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.295 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.297 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.298 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.298 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.299 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-08-29 16:32:23.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-29 16:32:23.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()